In [ ]:
import dask.dataframe as dd

# Login using e.g. `huggingface-cli login` to access this dataset
df = dd.read_parquet("hf://datasets/TomData/telegram-filtered-messages/data/train-*.parquet")

In [ ]:
df.head()

,sentence_id,sentence,channel_id,message_id,lang_mask,__index_level_0__
0,71ed70ba-60de-4495-a104-c10934f27297,The people who claim to hear the Northern Ligh...,1464907295,f295f749-e545-461d-a7b7-91ef67ed4ddd,en,8
1,9c78a102-56dc-4678-b2c0-e47fb7572d88,Tyr’s Appearance Has Huge Implications for God...,1464907295,e45c3d10-abcd-4894-97aa-778007df5a66,en,11
2,a563d4cb-67e2-49cf-a64d-1082911a4114,Europe’s oldest map shows tiny Bronze Age king...,1464907295,78df3bde-5263-4911-ad24-6d85705d4f74,en,13
3,e7274296-1103-4916-9713-56ea617a725f,Scientists discover ‘world’s northernmost isla...,1464907295,5e174b5b-783c-4e33-bc51-0c54b2292d85,en,15
4,8e91a3ab-006e-4a17-9578-55ad76e16e98,9 Bronze Age Weapons | HISTORY,1464907295,75120b28-d447-4de6-9213-25a4207f7a19,non-en,18


In [ ]:
df.shape

(<dask_expr.expr.Scalar: expr=ReadParquetFSSpec(df884b9).size() // 6, dtype=int64>,
 6)

In [ ]:
total_rows = df.shape[0].compute()
ten_percent = int(0.01 * total_rows)

In [ ]:
df_sample = df.head(ten_percent)

In [ ]:
df.info()

<class 'dask.dataframe.dask_expr.DataFrame'>
Columns: 6 entries, sentence_id to __index_level_0__
dtypes: int64(2), string(4)

In [ ]:
unique_sentence_ids = df_sample['sentence_id'].nunique()
print("Unique sentence IDs:", unique_sentence_ids)

Unique sentence IDs: 98849


In [ ]:
unique_lang_mask = df_sample['lang_mask'].nunique()
print("Unique sentence IDs:", unique_lang_mask)

Unique sentence IDs: 3


In [ ]:
unique_channel_ids = df_sample['channel_id'].nunique()
print("Unique channel IDs:", unique_channel_ids)

Unique channel IDs: 41


In [ ]:
unique_message_ids = df_sample['message_id'].nunique()
print("Unique message_id	s:", unique_message_ids)

Unique message_id	s: 53611


In [ ]:
df = df_sample.sort_values("message_id")

In [ ]:
chat_tokens = []

In [ ]:
chat_tokens.extend([
    {"token": "<|im_start|>"},
    "system\nYou are ChatGPT, a large language model trained by OpenAI. Answer as concisely as possible.\nKnowledge cutoff: 2021-09-01\nCurrent date: 2023-03-01",
    {"token": "<|im_end|>"},
    "\n"
])

In [ ]:
roles = ["user", "assistant"]

In [ ]:
for i, row in enumerate(df.itertuples()):
    role = roles[i % 2]
    sentence = str(row.sentence).strip()
    if not sentence:
        continue
    chat_tokens.extend([
        {"token": "<|im_start|>"},
        f"{role}\n{sentence}",
        {"token": "<|im_end|>"},
        "\n"
    ])

In [ ]:
import json

In [ ]:
with open("formatted_chat_tokens.json", "w", encoding="utf-8") as f:
    json.dump(chat_tokens, f, ensure_ascii=False, indent=2)